# Share geoDB Collections

The geoDB is a service provided by the [Euro Data Cube project](https://eurodatacube.com) (EDC) as a paid service. It comes
with a Python client that provides hugh level access to your data, and a certain amount of space in a PostGreSQL database. For managing (as sharing is)
your data you will need a management (read/write) account to your database which you can purchase at the [EDC market place](https://hub.eox.at).

You can access the service in two ways:

- By using the Jupyter Python notebook provided by EOX (configuration free, ```geodb = GeoDBClient()```)
- By using you own Jupyter notebook or Python script by providing a client id and secret to the GeoDBClient (```geodb = GeoDBClient(client_id="myid", client_secret="mysecet")```)

The client ID and secret is also provided by EOX in the latter case. You will find them in your EOX hub account section. You can also provide the credentials
via system environment variables (GEODB_AUTH_CLIENT_ID and GEODB_AUTH_CLIENT_SECRET). These variables can be supplied via a .env file.

There are two different types of geoDB accounts: a read only, and a management (read/write) access. The system will determine your access right through your
authentication credentials.


## Sharing Data

In [1]:
from xcube_geodb.core.geodb import GeoDBClient

##  Login from any maschine
### uncomment if not on EDC

Install xcube geoDB with command:
  
 `conda install xcube_geodb -c conda-forge`


In [2]:
#client_id=YourID
#client_secret=YourSecret
#geodb = GeoDBClient(client_id=client_id, client_secret=client_secret, auth_mode="client-credentials")

## Login in EDC environment
### comment if not on EDC

In [3]:
geodb = GeoDBClient()

### Get your user name

In [4]:
geodb.whoami

'geodb_ci_test_user'

In [6]:
import geopandas

# Have a look at fiona feature schema
collections = {
        "land_use":
        {
            "crs": 3794,
            "properties":
            {
                "RABA_PID": "float",
                "RABA_ID": "float",
                "D_OD": "date"
            }
        }
    }


geodb.create_collections(collections, clear=True)

gdf = geopandas.read_file('data/sample/land_use.shp')
geodb.insert_into_collection('land_use', gdf.iloc[:100,:]) # minimizing rows to 100, if you are in EDC, you dont need to make the subset.

Processing rows from 0 to 100


100 rows inserted into land_use

### Publish a Collection to the World

In [7]:
geodb.list_my_grants()

,Grants
0,No Grants


Please change the second positional kwargs to the geodb user you want to grant access to, if you are on EDC, go ahead and use 'geodb_test5' user:

In [8]:
geodb.grant_access_to_collection("land_use", "geodb_admin")

Access granted on land_use to geodb_admin

In [9]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_ci_test_user,land_use,geodb_admin,SELECT


### Accessing the Collection as a different User

Let's access the collection as a different user (a test user in this case), whose credentials were exported as an environment variable. You should now see a land_use collection.

In [10]:
import os

In [11]:
test_client_id = os.environ.get("TEST_CLIENT_ID")
test_client_secret = os.environ.get("TEST_CLIENT_SECRET")
test_client_geodb_api_server_url=os.environ.get("TEST_GEODB_API_SERVER_URL")

In [12]:
geodb = GeoDBClient(client_id=test_client_id, client_secret=test_client_secret, auth_mode="client-credentials", server_url=test_client_geodb_api_server_url)
geodb.whoami

'geodb_test5'

In [13]:
geodb.get_my_collections()

,owner,database,table_name
0,geodb_8c2d3fbe-f7a9-4492-8068-121e47e61a4f,test_db,test_default_db
1,geodb_8c2d3fbe-f7a9-4492-8068-121e47e61a4f,test_db,test_deleting
2,geodb_9bfgsdfg-453f-445b-a459,geodb_9bfgsdfg-453f-445b-a459,land_use


### Revoke access

Let's go back to the original user.

In [14]:
geodb = GeoDBClient()
geodb.whoami


'geodb_ci_test_user'

In [15]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_ci_test_user,land_use,geodb_admin,SELECT


In [16]:
geodb.revoke_access_from_collection("land_use", 'geodb_admin')

Access revoked from geodb_ci_test_user on land_use

In [17]:
geodb.list_my_grants()

,Grants
0,No Grants


### Finally going back to the initial user and deleting the collection:


In [20]:
geodb.drop_collection('land_use')


Collection ['geodb_ci_test_user_land_use'] deleted